In [2]:
# Imports
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import joblib

In [3]:
# Step 1: Load and Preprocess the Dataset
class FERDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        label = int(row['emotion'])
        image = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)  # FER2013 images are 48x48
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Load FER2013 data
df = pd.read_csv('fer20131.csv')

# Split into training and testing datasets
train_data, test_data = train_test_split(df, test_size=0.2, stratify=df['emotion'], random_state=42)

# Create PyTorch datasets
train_dataset = FERDataset(train_data)
test_dataset = FERDataset(test_data)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:

# Step 2: Load Pre-trained Model and Extract Features
model = models.resnet18(pretrained=True)
model.eval()  # Set to evaluation mode

# Adjust input channels for grayscale (FER2013 images are single-channel)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])  # Remove FC layer

def extract_features(dataloader, model):
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten the features
            features.append(outputs.numpy())
            labels.append(targets.numpy())
    return np.concatenate(features), np.concatenate(labels)

train_features, train_labels = extract_features(train_loader, feature_extractor)
test_features, test_labels = extract_features(test_loader, feature_extractor)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Step 3: Train SVM Classifier
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1, gamma='scale', random_state=42))
svm_classifier.fit(train_features, train_labels)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, random_state=42))])

In [6]:
# Step 4: Evaluate SVM Classifier
predictions = svm_classifier.predict(test_features)
print("Accuracy:", accuracy_score(test_labels, predictions))
print("Classification Report:")
print(classification_report(test_labels, predictions))

Accuracy: 0.3282251323488437
Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.09      0.13       991
           1       0.00      0.00      0.00       109
           2       0.26      0.08      0.12      1024
           3       0.33      0.76      0.46      1798
           4       0.32      0.25      0.28      1216
           5       0.38      0.27      0.32       800
           6       0.33      0.24      0.28      1240

    accuracy                           0.33      7178
   macro avg       0.27      0.24      0.23      7178
weighted avg       0.31      0.33      0.28      7178



/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [7]:
# Step 5: Save the SVM Model
# joblib.dump(svm_classifier, 'svm_model.pkl')